In [ ]:
def read_files(path):
    #function to read the training files and test file
    #the chosen encoding has proven to be optimal given the input data
    with open(path, 'rb',) as file:
        data = file.read().decode('utf8', 'surrogateescape')
        data = data.splitlines()
        #text_preprocess() removes punctuation, makes text all lowercase and
        #removes double/leading/trailing spaces
        data = text_preprocess(data)
    #make sure we drop empty lines
    data = list(filter(None, data))
    return data

In [1]:
import os

In [9]:
print(len(os.listdir('review_polarity.v2/txt_sentoken/neg/')))
print(len(os.listdir('review_polarity.v2/txt_sentoken/pos/')))

1000
1000


In [13]:
pos = []
for x in os.listdir('review_polarity.v2/txt_sentoken/pos/'):
    path = 'review_polarity.v2/txt_sentoken/pos/' + x
    with open(path)

review_polarity.v2/txt_sentoken/pos/cv839_21467.txt
review_polarity.v2/txt_sentoken/pos/cv034_29647.txt
review_polarity.v2/txt_sentoken/pos/cv908_16009.txt
review_polarity.v2/txt_sentoken/pos/cv748_12786.txt
review_polarity.v2/txt_sentoken/pos/cv253_10077.txt
review_polarity.v2/txt_sentoken/pos/cv147_21193.txt
review_polarity.v2/txt_sentoken/pos/cv962_9803.txt
review_polarity.v2/txt_sentoken/pos/cv686_13900.txt
review_polarity.v2/txt_sentoken/pos/cv410_24266.txt
review_polarity.v2/txt_sentoken/pos/cv913_29252.txt
review_polarity.v2/txt_sentoken/pos/cv695_21108.txt
review_polarity.v2/txt_sentoken/pos/cv601_23453.txt
review_polarity.v2/txt_sentoken/pos/cv490_17872.txt
review_polarity.v2/txt_sentoken/pos/cv518_13331.txt
review_polarity.v2/txt_sentoken/pos/cv157_29372.txt
review_polarity.v2/txt_sentoken/pos/cv570_29082.txt
review_polarity.v2/txt_sentoken/pos/cv289_6463.txt
review_polarity.v2/txt_sentoken/pos/cv098_15435.txt
review_polarity.v2/txt_sentoken/pos/cv656_24201.txt
review_polarit